In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 80.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 67.6 MB/s eta 0:00:0000:0100:01


In [1]:
import pandas as pd 
from sodapy import Socrata

In [2]:
client = Socrata("data.sfgov.org", None)

In [4]:
# results = client.get("wg3w-h783", limit=1000, offset= 0)

In [83]:
# condition = """
# report_datetime >= '2023-01-01T00:01:00.000' 
# AND
# report_datetime <= '2023-01-02T00:01:00.000'
# """

In [7]:
# This would create offset.txt file to keep record of # processed rows 

offset = ''
try:
    with open("offset.txt", "r") as f:
        offset = f.readline()
except FileNotFoundError:
    with open("offset.txt", "w+") as f:
        f.write("0")
        f.seek(0)
        offset = f.readline()
        
offset = int(offset)
offset

0

In [3]:
records = client.get("wg3w-h783", select="count(*)")

In [4]:
total_no_of_records = records[0]['count']

In [5]:
total_no_of_records = int(total_no_of_records)
total_no_of_records

714511

In [11]:
blob = pd.DataFrame()

In [12]:
while total_no_of_records != offset: 
    results = client.get("wg3w-h783", offset=offset, limit=50000)
    results_df = pd.DataFrame.from_records(results)

    # if blob.empty: 
    #     blob = results_df
    # else: 
    #     blob = blob.append(results_df, ignore_index=True)

    blob = pd.concat([results_df, blob])
    
    offset += len(results_df)
    offset

    with open("offset.txt", "w+") as f: 
        f.write(str(offset))

In [13]:
blob.shape

(714511, 34)

In [14]:
blob.head(1000).describe()

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,cad_number,...,longitude,point,:@computed_region_jwn9_ihcz,:@computed_region_26cr_cadq,:@computed_region_qgnn_b9vv,:@computed_region_n4xg_c4py,:@computed_region_nqbw_i6c3,filed_online,:@computed_region_h4ep_8xdi,:@computed_region_jg9y_a9du
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,493,...,567,567,560,567,566,55,103,389,55,7
unique,987,513,440,6,7,991,1000,997,996,490,...,472,472,95,11,10,1,5,1,1,1
top,2021-11-23T13:00:00.000,2021-05-26T00:00:00.000,00:00,2021,Friday,2021-11-23T13:00:00.000,103408063010,1032553,210541903,213283088,...,-122.43214039079034,"{'type': 'Point', 'coordinates': [-122.4321403...",32,10,4,1,3,True,1,1
freq,3,19,37,587,157,3,1,2,2,2,...,5,5,47,111,92,55,49,389,55,7


In [15]:
blob.head(4)

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,cad_number,...,longitude,point,:@computed_region_jwn9_ihcz,:@computed_region_26cr_cadq,:@computed_region_qgnn_b9vv,:@computed_region_n4xg_c4py,:@computed_region_nqbw_i6c3,filed_online,:@computed_region_h4ep_8xdi,:@computed_region_jg9y_a9du
0,2021-05-26T13:40:00.000,2021-05-26T00:00:00.000,13:40,2021,Wednesday,2021-05-26T13:41:00.000,103408063010,1034080,210324492,211461926,...,-122.38775628177875,"{'type': 'Point', 'coordinates': [-122.3877562...",56,9,2,NaN,NaN,NaN,NaN,NaN
1,2021-05-14T15:00:00.000,2021-05-14T00:00:00.000,15:00,2021,Friday,2021-05-15T13:55:00.000,103047707021,1030477,210298665,211351630,...,-122.4347299934904,"{'type': 'Point', 'coordinates': [-122.4347299...",103,11,4,NaN,NaN,NaN,NaN,NaN
2,2021-05-24T18:00:00.000,2021-05-24T00:00:00.000,18:00,2021,Monday,2021-05-24T20:31:00.000,103377506242,1033775,210320616,211443014,...,-122.41580242607432,"{'type': 'Point', 'coordinates': [-122.4158024...",99,3,6,NaN,NaN,NaN,NaN,NaN
3,2021-05-22T09:45:00.000,2021-05-22T00:00:00.000,09:45,2021,Saturday,2021-05-22T16:14:00.000,103286974000,1032869,210315388,211421801,...,-122.40838011465424,"{'type': 'Point', 'coordinates': [-122.4083801...",32,10,1,1,1,NaN,NaN,NaN


In [16]:
from pathlib import Path

In [18]:
# Extract year, month, and date from the incident_date column
# blob['year'] = pd.DatetimeIndex(blob['incident_date']).year.astype(str)

col = pd.DatetimeIndex(blob['incident_date']).month.astype(str).str.zfill(2)
blob.insert(loc=4,column='incident_month',value=col )

col = pd.DatetimeIndex(blob['incident_date']).day.astype(str).str.zfill(2)
blob.insert(loc=5, column="incident_day", value=col)



ValueError: cannot insert incident_month, already exists

In [20]:
blob['incident_year'] = blob['incident_year'].astype(str)

In [21]:
blob.head()

,incident_datetime,incident_date,incident_time,incident_year,incident_month,incident_day,incident_day_of_week,report_datetime,row_id,incident_id,...,longitude,point,:@computed_region_jwn9_ihcz,:@computed_region_26cr_cadq,:@computed_region_qgnn_b9vv,:@computed_region_n4xg_c4py,:@computed_region_nqbw_i6c3,filed_online,:@computed_region_h4ep_8xdi,:@computed_region_jg9y_a9du
0,2021-05-26T13:40:00.000,2021-05-26T00:00:00.000,13:40,2021,05,26,Wednesday,2021-05-26T13:41:00.000,103408063010,1034080,...,-122.38775628177875,"{'type': 'Point', 'coordinates': [-122.3877562...",56,9,2,NaN,NaN,NaN,NaN,NaN
1,2021-05-14T15:00:00.000,2021-05-14T00:00:00.000,15:00,2021,05,14,Friday,2021-05-15T13:55:00.000,103047707021,1030477,...,-122.4347299934904,"{'type': 'Point', 'coordinates': [-122.4347299...",103,11,4,NaN,NaN,NaN,NaN,NaN
2,2021-05-24T18:00:00.000,2021-05-24T00:00:00.000,18:00,2021,05,24,Monday,2021-05-24T20:31:00.000,103377506242,1033775,...,-122.41580242607432,"{'type': 'Point', 'coordinates': [-122.4158024...",99,3,6,NaN,NaN,NaN,NaN,NaN
3,2021-05-22T09:45:00.000,2021-05-22T00:00:00.000,09:45,2021,05,22,Saturday,2021-05-22T16:14:00.000,103286974000,1032869,...,-122.40838011465424,"{'type': 'Point', 'coordinates': [-122.4083801...",32,10,1,1,1,NaN,NaN,NaN
4,2019-11-01T00:00:00.000,2019-11-01T00:00:00.000,00:00,2019,11,01,Friday,2019-11-25T06:52:00.000,87508071000,875080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN


In [23]:
blob.dtypes

incident_datetime              object
incident_date                  object
incident_time                  object
incident_year                  object
incident_month                 object
incident_day                   object
incident_day_of_week           object
report_datetime                object
row_id                         object
incident_id                    object
incident_number                object
cad_number                     object
report_type_code               object
report_type_description        object
incident_code                  object
incident_category              object
incident_subcategory           object
incident_description           object
resolution                     object
intersection                   object
cnn                            object
police_district                object
analysis_neighborhood          object
supervisor_district            object
supervisor_district_2012       object
latitude                       object
longitude   

In [15]:
# Create directory structure if it doesn't exist
for year in blob['incident_year'].unique():
    year_dir = Path('data') / year
    year_dir.mkdir(parents=True, exist_ok=True)
    for month in blob[blob['incident_year'] == year]['incident_month'].unique():
        month_dir = year_dir / month
        month_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Save the DataFrame into year/month/day CSV files
for _, row in blob.iterrows():
    file_path = Path('data') / row['incident_year'] / row['incident_month'] / f"{row['incident_day']}.csv"
    row.to_frame().T.to_csv(file_path, mode='a', header=not file_path.exists(), index=False)

In [28]:
import pyarrow as pa

# Define the schema
schema = pa.schema([
    ('incident_datetime', pa.string()),
    ('incident_date', pa.string()),
    ('incident_time', pa.string()),
    ('incident_year', pa.string()),
    ('incident_month', pa.string()),
    ('incident_day', pa.string()),
    ('incident_day_of_week', pa.string()),
    ('report_datetime', pa.string()),
    ('row_id', pa.string()),
    ('incident_id', pa.string()),
    ('incident_number', pa.string()),
    ('report_type_code', pa.string()),
    ('report_type_description', pa.string()),
    ('incident_code', pa.string()),
    ('incident_category', pa.string()),
    ('incident_subcategory', pa.string()),
    ('incident_description', pa.string()),
    ('resolution', pa.string()),
    ('police_district', pa.string()),
    ('filed_online', pa.string()),
    ('cad_number', pa.string()),
    ('intersection', pa.string()),
    ('cnn', pa.string()),
    ('analysis_neighborhood', pa.string()),
    ('supervisor_district', pa.string()),
    ('supervisor_district_2012', pa.string()),
    ('latitude', pa.string()),
    ('longitude', pa.string()),
    ('point', pa.string()),
    (':@computed_region_26cr_cadq', pa.string()),
    (':@computed_region_qgnn_b9vv', pa.string()),
    (':@computed_region_jwn9_ihcz', pa.string()),
    (':@computed_region_h4ep_8xdi', pa.string()),
    (':@computed_region_nqbw_i6c3', pa.string()),
    (':@computed_region_n4xg_c4py', pa.string()),
    (':@computed_region_jg9y_a9du', pa.string())
])

# Print the schema
print(schema)


incident_datetime: string
incident_date: string
incident_time: string
incident_year: string
incident_month: string
incident_day: string
incident_day_of_week: string
report_datetime: string
row_id: string
incident_id: string
incident_number: string
report_type_code: string
report_type_description: string
incident_code: string
incident_category: string
incident_subcategory: string
incident_description: string
resolution: string
police_district: string
filed_online: string
cad_number: string
intersection: string
cnn: string
analysis_neighborhood: string
supervisor_district: string
supervisor_district_2012: string
latitude: string
longitude: string
point: string
:@computed_region_26cr_cadq: string
:@computed_region_qgnn_b9vv: string
:@computed_region_jwn9_ihcz: string
:@computed_region_h4ep_8xdi: string
:@computed_region_nqbw_i6c3: string
:@computed_region_n4xg_c4py: string
:@computed_region_jg9y_a9du: string


In [29]:
# Save the DataFrame into yeartest/2020/01/01.snappy.parquetpy.parquet files

# Doestn't work 


for _, row in blob.iterrows():
    file_path = Path('test') / str(row['incident_year']) / str(row['incident_month']) / f"{row['incident_day']}.snappy.parquet"
    df = row.to_frame().T
    table = pa.Table.from_pandas(df, schema=schema)
    print(table)
    break
    # if file_path.is_file():
    #     pq.write_table(table, 'example.parquet', append=True,)
    # else:
    #     pq.write_table(table, 'example.parquet', )
        

ArrowTypeError: ("Expected bytes, got a 'dict' object", 'Conversion failed for column point with type object')

In [20]:
df_parquet = pd.read_parquet('test/2020/02/08.snappy.parquet')

In [21]:
df_parquet.head()

,incident_datetime,incident_date,incident_time,incident_year,incident_month,incident_day,incident_day_of_week,report_datetime,row_id,incident_id,...,latitude,longitude,point,:@computed_region_26cr_cadq,:@computed_region_qgnn_b9vv,:@computed_region_jwn9_ihcz,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_n4xg_c4py,:@computed_region_jg9y_a9du
0,2020-02-08T16:45:00.000,2020-02-08T00:00:00.000,16:45,2020,02,08,Saturday,2020-02-11T12:59:00.000,90225906374,902259,...,None,None,None,None,None,None,None,None,None,None


In [90]:
df = pd.read_csv('data/2020/01/01.csv')

In [91]:
df.to_parquet('test.snappy.parquet', compression='snappy', index=False)

In [92]:
df = pd.read_parquet('test.snappy.parquet')

In [94]:
df.head()

,incident_datetime,incident_date,incident_time,incident_year,incident_month,incident_day,incident_day_of_week,report_datetime,row_id,incident_id,...,latitude,longitude,point,:@computed_region_jwn9_ihcz,:@computed_region_jg9y_a9du,:@computed_region_26cr_cadq,:@computed_region_qgnn_b9vv,:@computed_region_nqbw_i6c3,:@computed_region_h4ep_8xdi,:@computed_region_n4xg_c4py
0,2020-01-01T21:00:00.000,2020-01-01T00:00:00.000,21:00,2020,1,1,Wednesday,2020-01-01T21:00:00.000,88706007041,887060,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01T00:01:00.000,2020-01-01T00:00:00.000,00:01,2020,1,1,Wednesday,2021-06-10T11:37:00.000,103882213045,1038822,...,37.783101,-122.419182,"{'type': 'Point', 'coordinates': [-122.4191817...",20.0,NaN,10.0,4.0,NaN,1.0,1.0
2,2020-01-01T08:00:00.000,2020-01-01T00:00:00.000,08:00,2020,1,1,Wednesday,2021-05-17T09:27:00.000,103086406374,1030864,...,37.740898,-122.384706,"{'type': 'Point', 'coordinates': [-122.3847055...",86.0,NaN,9.0,2.0,NaN,NaN,NaN
3,2020-01-01T00:01:00.000,2020-01-01T00:00:00.000,00:01,2020,1,1,Wednesday,2021-06-10T11:37:00.000,103882230011,1038822,...,37.783101,-122.419182,"{'type': 'Point', 'coordinates': [-122.4191817...",20.0,NaN,10.0,4.0,NaN,1.0,1.0
4,2020-01-01T00:30:00.000,2020-01-01T00:00:00.000,00:30,2020,1,1,Wednesday,2020-01-17T21:22:00.000,89300006374,893000,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Check for recently modified files

from datetime import datetime, timedelta
from pathlib import Path


In [40]:
# Specify the directory to search for files in
directory_path = Path('data/')

# Calculate the datetime range for today
today = datetime.today()
start_time = datetime(today.year, today.month, today.day)
end_time = start_time + timedelta(days=1)

<generator object Path.glob at 0x7f88cd9f3c10>

In [44]:
# Loop through all files in the directory and get the paths of the ones modified today

for file_path in directory_path.glob('**/*'):
        if file_path.is_file():
            mtime = datetime.fromtimestamp(file_path.stat().st_mtime)
            if start_time <= mtime < end_time and "ipynb_checkpoints" not in str(file_path):
                print(file_path)
                

data/2023/03/23.csv
data/2023/03/03.csv
data/2023/03/16.csv
data/2023/03/12.csv
data/2023/03/15.csv
data/2023/03/30.csv
data/2023/03/10.csv
data/2023/03/22.csv
data/2023/03/09.csv
data/2023/03/04.csv
data/2023/03/20.csv
data/2023/03/08.csv
data/2023/03/21.csv
data/2023/03/24.csv
data/2023/03/11.csv
data/2023/03/01.csv
data/2023/03/05.csv
data/2023/03/26.csv
data/2023/03/27.csv
data/2023/03/29.csv
data/2023/03/13.csv
data/2023/03/31.csv
data/2023/03/18.csv
data/2023/03/14.csv
data/2023/03/19.csv
data/2023/03/17.csv
data/2023/03/28.csv
data/2023/03/07.csv
data/2023/03/02.csv
data/2023/03/06.csv
data/2023/03/25.csv
data/2023/04/01.csv
data/2023/04/02.csv
data/2023/02/23.csv
data/2023/02/03.csv
data/2023/02/16.csv
data/2023/02/12.csv
data/2023/02/15.csv
data/2023/02/10.csv
data/2023/02/22.csv
data/2023/02/09.csv
data/2023/02/04.csv
data/2023/02/20.csv
data/2023/02/08.csv
data/2023/02/21.csv
data/2023/02/24.csv
data/2023/02/11.csv
data/2023/02/01.csv
data/2023/02/05.csv
data/2023/02/26.csv


In [61]:
from prefect_gcp.cloud_storage import GcsBucket

In [46]:
new_bucket = GcsBucket(bucket='my_new_bucket')


In [47]:
new_bucket.create_bucket()  # doesn't work i guess

<coroutine object GcsBucket.create_bucket at 0x7f88c6516a40>

In [62]:
path = Path('data/2023/03/23.csv')
gcs_block = await GcsBucket.load("gcp-bucket-block")
await gcs_block.upload_from_path(from_path=path, to_path=path)

06:28:15.004 | INFO    | prefect.GcsBucket - Getting bucket 'incident_data_lake_affable-tangent-382517'.

ImportError: To use prefect_gcp.cloud_storage, install prefect-gcp with the 'cloud_storage' extra: `pip install 'prefect_gcp[cloud_storage]'`

In [82]:
results_df = pd.DataFrame.from_records(results)

In [80]:
with open("offset.txt", "w+") as f: 
    f.write(str(offset))

In [88]:
results = client.get("wg3w-h783", offset=offset,order="report_datetime")

In [84]:
results = client.get("wg3w-h783", where=condition)

In [90]:
results_df

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,cad_number,...,resolution,intersection,cnn,police_district,analysis_neighborhood,supervisor_district,supervisor_district_2012,latitude,longitude,point
0,2023-03-30T18:00:00.000,2023-03-30T00:00:00.000,18:00,2023,Thursday,2023-04-02T21:10:00.000,126170906241,1261709,230229076,230922935,...,Open or Active,16TH ST \ JOSE SARRIA CT \ POND ST,25800000,Mission,Castro/Upper Market,8,8,37.76428896385761,-122.43227951733411,"{'type': 'Point', 'coordinates': [-122.4322795..."
1,2023-04-02T14:25:00.000,2023-04-02T00:00:00.000,14:25,2023,Sunday,2023-04-02T21:35:00.000,126171128161,1261711,230226486,230922904,...,Open or Active,PALOU AVE \ HAWES ST,20118000,Bayview,Bayview Hunters Point,10,10,37.72867801184137,-122.38156441612668,"{'type': 'Point', 'coordinates': [-122.3815644..."
2,2023-04-02T14:25:00.000,2023-04-02T00:00:00.000,14:25,2023,Sunday,2023-04-02T21:35:00.000,126171128160,1261711,230226486,230922904,...,Open or Active,PALOU AVE \ HAWES ST,20118000,Bayview,Bayview Hunters Point,10,10,37.72867801184137,-122.38156441612668,"{'type': 'Point', 'coordinates': [-122.3815644..."
3,2023-04-02T21:37:00.000,2023-04-02T00:00:00.000,21:37,2023,Sunday,2023-04-02T21:37:00.000,126171971012,1261719,230216631,230872414,...,Open or Active,BRYANT ST \ LANGTON ST,23876000,Southern,South of Market,6,6,37.77385630397913,-122.40528580238285,"{'type': 'Point', 'coordinates': [-122.4052858..."
4,2023-04-02T21:38:00.000,2023-04-02T00:00:00.000,21:38,2023,Sunday,2023-04-02T21:38:00.000,126171707043,1261717,230125004,NaN,...,Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-04-02T22:35:00.000,2023-04-02T00:00:00.000,22:35,2023,Sunday,2023-04-02T22:35:00.000,126173007055,1261730,230229195,230923119,...,Open or Active,04TH ST \ BRANNAN ST,23827000,Southern,South of Market,6,6,37.7783268304671,-122.39653063923612,"{'type': 'Point', 'coordinates': [-122.3965306..."
6,2023-04-02T22:41:00.000,2023-04-02T00:00:00.000,22:41,2023,Sunday,2023-04-02T22:41:00.000,126141264020,1261412,230227020,230913295,...,Open or Active,ARBALLO DR \ SERRANO DR,23098000,Taraval,Lakeshore,7,7,37.719109027120155,-122.48310424113126,"{'type': 'Point', 'coordinates': [-122.4831042..."


Below things should be considered when pushing to gcs in parquet

In [28]:
results_filtered = results_df[results_df[':@computed_region_nqbw_i6c3'].notnull()]

In [38]:
results_filtered.head(5)

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,report_type_code,...,supervisor_district_2012,latitude,longitude,point,:@computed_region_26cr_cadq,:@computed_region_qgnn_b9vv,:@computed_region_jwn9_ihcz,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_n4xg_c4py
220,2021-07-12 18:18:00,2021-07-12,18:18:00,2021,Monday,2021-07-12T18:26:00.000,104984326080,1049843,210440565,II,...,9,37.74817939136343,-122.41917060471224,"{'type': 'Point', 'coordinates': [-122.4191706...",2,3,53,NaN,3,NaN
221,2021-07-07 08:18:00,2021-07-07,08:18:00,2021,Wednesday,2021-07-07T08:28:00.000,104785804134,1047858,210426383,II,...,9,37.753836586731786,-122.41859394629981,"{'type': 'Point', 'coordinates': [-122.4185939...",2,3,53,NaN,3,NaN
304,2021-07-12 12:20:00,2021-07-12,12:20:00,2021,Monday,2021-07-12T15:20:00.000,104980919057,1049809,210440703,II,...,6,37.78321431177312,-122.41076482950653,"{'type': 'Point', 'coordinates': [-122.4107648...",10,5,20,1,1,1
309,2019-06-11 16:30:00,2019-06-11,16:30:00,2019,Tuesday,2021-06-05T05:17:00.000,103770002004,1037700,210348157,II,...,6,37.775953228502786,-122.40884575063417,"{'type': 'Point', 'coordinates': [-122.4088457...",10,1,32,NaN,1,1
342,2022-09-25 17:00:00,2022-09-25,17:00:00,2022,Sunday,2022-09-26T17:09:00.000,119741374000,1197413,220659671,II,...,9,37.75723240847821,-122.41562619847495,"{'type': 'Point', 'coordinates': [-122.4156261...",2,3,53,NaN,3,NaN


In [33]:
results_filtered['incident_datetime'] = results_filtered['incident_datetime'].astype('datetime64[ns]')

/tmp/ipykernel_11518/2783132438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_filtered['incident_datetime'] = results_filtered['incident_datetime'].astype('datetime64[ns]')


In [34]:
results_filtered['incident_date'] = results_filtered['incident_date'].astype('datetime64[ns]')

/tmp/ipykernel_11518/4139641677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_filtered['incident_date'] = results_filtered['incident_date'].astype('datetime64[ns]')


In [35]:
results_filtered['incident_time'] = pd.to_datetime(results_filtered['incident_time']).dt.time

TypeError: <class 'datetime.time'> is not convertible to datetime, at position 0

In [40]:
results_filtered['incident_year'] = results_filtered['incident_year'].astype(int) 

/tmp/ipykernel_11518/762521036.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_filtered['incident_year'] = results_filtered['incident_year'].astype(int)


In [43]:
results_filtered.to_parquet('test.parquet')

In [44]:
par_df = pd.read_parquet('test.parquet')

In [47]:
par_df.dtypes

incident_datetime              datetime64[ns]
incident_date                  datetime64[ns]
incident_time                          object
incident_year                           int64
incident_day_of_week                   object
report_datetime                        object
row_id                                 object
incident_id                            object
incident_number                        object
report_type_code                       object
report_type_description                object
incident_code                          object
incident_category                      object
incident_subcategory                   object
incident_description                   object
resolution                             object
police_district                        object
filed_online                           object
cad_number                             object
intersection                           object
cnn                                    object
analysis_neighborhood             